<a href="https://colab.research.google.com/github/marcelowender/fumec/blob/master/TrabalhoFinalText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho final Processamento de Linguagem Natural


Modelagem de tópicos realizada utilizando uma base de dados de descrições de cargos de diversas empresas de vários segmentos.

Através do processo de modelagem de tópicos este trabalho tem como objetivo analisar os dados desta base e encontrar padrões.

**Descrição de Cargos**


A descrição de cargos é o processo de detalhamento de cada posição dentro da companhia. Com ela, é possível verificar e formalizar as atividades e responsabilidades de todos os postos da organização, agregando maior organização ao negócio.

## Import das bibliotecas

Foram utilizadas as seguintes bibliotecas:




> **json** - Responsável pela leitura do corpus 

> **drive** - A biblioteca drive é utilizada para carregar um arquivo a partir do Google Drive


> **spacy** - A biblioteca spacy ajuda no pre processamento do texto e na preparação do corpus


> **gensim** - Biblioteca que possui os algoritimos para realizar a modelagem de tópicos



> **nltk** - Biblioteca utilizada também no pré processamento  do texto(remoção de stopwords e tokenização)



In [0]:
import json
import spacy
import gensim
import nltk
from google.colab import drive
!python -m spacy download pt_core_news_sm
nlp = spacy.load("pt_core_news_sm")
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


    100% |████████████████████████████████| 38.7MB 120.7MB/s 
  Running setup.py install for pt-core-news-sm ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/pt_core_news_sm

    You can now load the model via spacy.load('pt_core_news_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


### **Montagem do Drive**

In [0]:
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Leitura do arquivo

In [0]:
with open('/content/drive/My Drive/Cargos.json') as f:
  cargos = json.loads(f.read(), strict=False)


### Visualização dos dados

In [0]:
print(*cargos[:9], sep='\n') 

#https://docs.python.org/3.7/tutorial/controlflow.html#unpacking-argument-lists

{'enterprise': 2, 'cbo': '142315', 'name': 'Gerente de Marketing', 'summarydescription': 'Gerenciar a área de marketing.', 'description': '["Realizar ,2", "Realizar ! 1"]', 'attribute': '["Marketing", "Análise de Sistemas", "Inglês", "Espanhol", "Gestão de pessoas"]'}
{'enterprise': 3, 'cbo': '142205', 'name': 'Gerente de Recursos Humanos', 'summarydescription': 'Gerenciar, planejar, controlar o processo de Recrutamento e Seleção de pessoal, ocorra de acordo com os perfis indicados pelas áreas, a implementação, manutenção e cumprimento dos planos e políticas de remuneração, benefícios, relações trabalhistas, carreira e sucessão e acompanhamento de pessoal.', 'description': None, 'attribute': None}
{'enterprise': 3, 'cbo': '123205', 'name': 'Diretor de Recursos Humanos', 'summarydescription': '', 'description': None, 'attribute': '["Análise de Sistemas", "Antropologia", "Arqueologia", "Francês", "Análise de problemas"]'}
{'enterprise': 4, 'cbo': None, 'name': 'Analista de Recursos Human

### Limpeza dos dados

Removendo cargos sem descrições

In [0]:
cargosLimpos = []
for i in range(len(cargos)):
  cargo = cargos[i]
  if cargo['summarydescription'] is not None and cargo['summarydescription'].strip() != '':
    cargosLimpos.append(cargo)

print(len(cargosLimpos))
print(len(cargos))

5584
5616


### Agrupamento dos textos. 
Concatenada a descrição sumária, e adescrição detalhada

In [0]:
listaCargos = []
for i in range(len(cargosLimpos)):
  cargo = cargosLimpos[i]
  textoCargo = ''
  if cargo['summarydescription'] is not None:
   textoCargo = cargo['summarydescription'].lower()
  if cargo['description'] is not None:
   description = json.loads(cargo['description'])
   textoCargo += " ".join(description)
 
  listaCargos.append(textoCargo)


Visualização dos dados após o agrupamento dos textos.

In [0]:

print(*listaCargos[:9], sep='\n')

gerenciar a área de marketing.Realizar ,2 Realizar ! 1
gerenciar, planejar, controlar o processo de recrutamento e seleção de pessoal, ocorra de acordo com os perfis indicados pelas áreas, a implementação, manutenção e cumprimento dos planos e políticas de remuneração, benefícios, relações trabalhistas, carreira e sucessão e acompanhamento de pessoal.
analisar, controlar e acompanhar a execução das atividades de administração de pessoal, folha de pagamento, processos de admissão, férias e rescisão e na elaboração de relatórios diversos.Atuar no desenvolvimento e aplicação de novos projetos e processos, assegurando sua devida implantação. Cumprir a legislação e políticas da Empresa, bem como os padrões e procedimentos de saúde, segurança e meio ambiente, zelar pelas condições de higiene e organização no ambiente de trabalho. Analisar, aplicar, controlar e acompanhar as atividades e recursos inerentes a área de Recursos Humanos da Empresa. Acompanhar o processo de auditoria interna nos p

### Remoção de dados
Remoção de Stop words 

In [0]:
stop_words = stopwords.words('portuguese')

listaCargosSemStopWords = []
for i in range(len(listaCargos)):
   palavras = word_tokenize(listaCargos[i])
   valor = ""
   for palavra in palavras:
      if palavra not in stop_words:
        valor += palavra
      valor += " " 
   listaCargosSemStopWords.append(nlp(valor))

In [0]:
print(*listaCargosSemStopWords[:20], sep='\n')

gerenciar  área  marketing.Realizar ,2 Realizar ! 1 
gerenciar , planejar , controlar  processo  recrutamento  seleção  pessoal , ocorra  acordo   perfis indicados  áreas ,  implementação , manutenção  cumprimento  planos  políticas  remuneração , benefícios , relações trabalhistas , carreira  sucessão  acompanhamento  pessoal . 
analisar , controlar  acompanhar  execução  atividades  administração  pessoal , folha  pagamento , processos  admissão , férias  rescisão   elaboração  relatórios diversos.Atuar  desenvolvimento  aplicação  novos projetos  processos , assegurando  devida implantação . Cumprir  legislação  políticas  Empresa , bem   padrões  procedimentos  saúde , segurança  meio ambiente , zelar  condições  higiene  organização  ambiente  trabalho . Analisar , aplicar , controlar  acompanhar  atividades  recursos inerentes  área  Recursos Humanos  Empresa . Acompanhar  processo  auditoria interna  procedimentos  SOX . Assegurar  cumprimento  legislação , bem   normas  polític

Remoção de Pontuação

In [0]:

texts, article = [], []
for cargo in listaCargosSemStopWords:
  for w in cargo:
    if not w.is_space and not w.is_punct:
      article.append(w.lemma_)
  texts.append(article)
  article = []
          

In [0]:
bigram = gensim.models.Phrases(texts)


In [0]:
texts = [bigram[line] for line in texts]

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
texts[7][0:10]

['elaborar',
 'relatório_gerenciais',
 'acompanhamento',
 'desempenhar',
 'área',
 'negócio',
 'dar',
 'suportar',
 'área',
 'não']

### Bag of Words

COnvertendo os  documentos tokenizados em uma matriz de termo de documento

In [0]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [0]:

corpus[100]

[(2, 1),
 (5, 2),
 (7, 1),
 (9, 3),
 (10, 3),
 (18, 1),
 (19, 1),
 (24, 1),
 (27, 1),
 (28, 1),
 (31, 7),
 (32, 1),
 (33, 1),
 (34, 8),
 (35, 1),
 (36, 1),
 (41, 4),
 (44, 1),
 (47, 1),
 (48, 1),
 (53, 1),
 (55, 1),
 (57, 2),
 (58, 1),
 (59, 1),
 (60, 1),
 (63, 1),
 (64, 1),
 (65, 2),
 (66, 1),
 (67, 2),
 (68, 1),
 (75, 4),
 (76, 2),
 (77, 1),
 (79, 1),
 (80, 2),
 (82, 3),
 (88, 1),
 (94, 1),
 (95, 1),
 (99, 2),
 (101, 2),
 (102, 2),
 (103, 4),
 (104, 1),
 (105, 2),
 (107, 2),
 (108, 5),
 (110, 1),
 (112, 2),
 (114, 4),
 (115, 1),
 (116, 1),
 (121, 1),
 (123, 1),
 (124, 1),
 (127, 1),
 (128, 1),
 (131, 2),
 (132, 1),
 (133, 2),
 (134, 3),
 (136, 3),
 (137, 1),
 (138, 3),
 (143, 4),
 (147, 2),
 (148, 2),
 (149, 1),
 (158, 1),
 (188, 2),
 (199, 1),
 (265, 4),
 (268, 1),
 (291, 1),
 (298, 1),
 (304, 1),
 (308, 1),
 (310, 1),
 (313, 2),
 (314, 1),
 (315, 1),
 (317, 1),
 (331, 1),
 (337, 1),
 (388, 1),
 (397, 1),
 (402, 1),
 (404, 1),
 (438, 1),
 (467, 1),
 (514, 1),
 (567, 1),
 (594, 1),
 

### Latent semantic indexing (LSI)

É um método de indexação e recuperação que utiliza uma técnica matemática chamada decomposição de valor singular (SVD) para identificar padrões nas relações entre os termos e conceitos contidos em uma coleção não estruturada de texto.

In [0]:
lsimodel = LsiModel(corpus=corpus, num_topics=5, id2word=dictionary)

### Topicos Gerados

In [0]:
lsimodel.show_topics(num_topics=5)

[(0,
  '0.333*"visar" + 0.318*"Empresa" + 0.272*"procedimento" + 0.213*"cumprimentar" + 0.191*"norma" + 0.183*"interno" + 0.180*"bem" + 0.177*"área" + 0.174*"acordar" + 0.174*"garantir"'),
 (1,
  '-0.334*"segurança" + -0.274*"norma" + 0.256*"definir" + 0.238*"cumprimentar" + -0.209*"trabalhar" + 0.202*"meter" + -0.159*"equipamento" + 0.154*"visar" + 0.153*"Garantir" + -0.151*"saudar"'),
 (2,
  '-0.407*"acordar" + -0.377*"interno" + -0.334*"procedimento" + -0.269*"definir" + 0.244*"fim" + -0.191*"visar" + -0.148*"vigente" + 0.134*"garantir" + 0.132*"trabalhar" + 0.131*"área"'),
 (3,
  '-0.299*"setor" + 0.252*"segurança" + -0.213*"Empresa" + 0.212*"qualidade" + 0.208*"equipamento" + 0.202*"norma" + 0.194*"manutenção" + -0.184*"processo" + -0.164*"procedimento" + 0.162*"garantir"'),
 (4,
  '-0.506*"Empresa" + 0.230*"área" + 0.183*"visar" + 0.164*"informação" + 0.156*"cliente" + -0.152*"cumprimentar" + -0.133*"saudar" + 0.130*"equipamento" + 0.122*"técnico" + -0.122*"definir"')]

### Latent Dirichlet Allocation (LDA) 
É um exemplo de modelo de tópico e é usado para classificar o texto em um documento para um tópico específico.

In [0]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)

### Topicos Gerados

In [0]:
ldamodel.show_topics()

[(0,
  '0.015*"Empresa" + 0.014*"visar" + 0.013*"área" + 0.012*"trabalhar" + 0.012*"fim" + 0.012*"norma" + 0.010*"serviço" + 0.010*"segurança" + 0.009*"garantir" + 0.008*"atividades"'),
 (1,
  '0.014*"fim" + 0.013*"norma" + 0.012*"garantir" + 0.011*"segurança" + 0.011*"área" + 0.009*"bem" + 0.009*"visar" + 0.009*"acompanhar" + 0.009*"trabalhar" + 0.008*"Empresa"'),
 (2,
  '0.024*"norma" + 0.024*"segurança" + 0.019*"procedimento" + 0.019*"visar" + 0.018*"Empresa" + 0.016*"acordar" + 0.016*"interno" + 0.013*"trabalhar" + 0.010*"bem" + 0.010*"padrão"'),
 (3,
  '0.013*"manutenção" + 0.013*"equipamento" + 0.012*"procedimento" + 0.011*"visar" + 0.011*"segurança" + 0.010*"técnico" + 0.009*"bem" + 0.008*"trabalhar" + 0.007*"representar_risco" + 0.007*"Empresa"'),
 (4,
  '0.025*"visar" + 0.021*"procedimento" + 0.019*"cumprimentar" + 0.018*"Empresa" + 0.015*"trabalhar" + 0.014*"norma" + 0.014*"segurança" + 0.012*"interno" + 0.012*"acordar" + 0.012*"garantir"'),
 (5,
  '0.018*"Empresa" + 0.016*"á